### [Special method lookup for old-style classes](https://docs.python.org/2/reference/datamodel.html#special-method-lookup-for-old-style-classes)


    For old-style classes, special methods are always looked up in exactly the same way as any other method or attribute. This is the case regardless of whether the method is being looked up explicitly as in x.__getitem__(i) or implicitly as in x[i].


    This behaviour means that special methods may exhibit different behaviour for different instances of a single old-style class if the appropriate special attributes are set differently:

In [3]:
class C:
    pass

c1 = C()
c2 = C()
c1.__len__ = lambda: 5
c2.__len__ = lambda: 9

print len(c1), len(c2)

5 9


### [Special method lookup for new-style classes](https://docs.python.org/2/reference/datamodel.html#special-method-lookup-for-new-style-classes)

    For new-style classes, implicit invocations of special methods are only guaranteed to work correctly if defined on an object’s type, not in the object’s instance dictionary. 

    That behaviour is the reason why the following code raises an exception 
    (unlike the equivalent example with old-style classes):


In [4]:
class C(object):
    pass

c = C()
c.__len__ = lambda: 5
len(c)

TypeError: object of type 'C' has no len()

In [9]:
class C(object):
    def __len__(self): return 3
    pass

c = C()
c.__len__ = lambda: 5
len(c)

3


The rationale behind this behaviour lies with a number of special methods such as `__hash__() and __repr__()` that are implemented by all objects, including type objects. If the implicit lookup of these methods used the conventional lookup process, they would fail when invoked on the type object itself:


In [13]:
print 1 .__hash__() == hash(1)

print int.__hash__() == hash(int)

True


TypeError: descriptor '__hash__' of 'int' object needs an argument

Incorrectly attempting to invoke an unbound method of a class in this way is sometimes referred to as ‘metaclass confusion’, and is avoided by bypassing the instance when looking up special methods:

In [12]:
print type(1).__hash__(1) == hash(1)

print type(int).__hash__(int) == hash(int)

True
True


In addition to bypassing any instance attributes in the interest of correctness, **implicit** special method lookup generally also bypasses the `__getattribute__()` method even of the object’s metaclass:


In [16]:
class Meta(type):
   def __getattribute__(*args):
      print "Metaclass getattribute invoked"
      return type.__getattribute__(*args)

class C(object):
    __metaclass__ = Meta
    def __len__(self):
        return 10
    def __getattribute__(*args):
        print "Class getattribute invoked"
        return object.__getattribute__(*args)

c = C()
print c.__len__()                 # Explicit lookup via instance
print type(c).__len__(c)          # Explicit lookup via type
print len(c)                      # Implicit lookup

Class getattribute invoked
10
Metaclass getattribute invoked
10
10


Bypassing the `__getattribute__()` machinery in this fashion provides significant scope for speed optimisations within the interpreter, at the cost of some flexibility in the handling of special methods (the special method must be set on the class object itself in order to be consistently invoked by the interpreter).